In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import illustris_python as il

In [28]:
with h5py.File('/home/group10/TNGDataScripts/TNG50-4-Dark/output/groups_099/fof_subhalo_tab_099.0.hdf5','r') as hdf:
    ls = list(hdf.keys())
    print("list of datasets in this file: \n", ls)
    data = hdf.get('Subhalo')
    dataset1 = np.array(data)
    subhalo_items = list(data.items())
    print("items if subhalos",subhalo_items)

list of datasets in this file: 
 ['Config', 'Group', 'Header', 'IDs', 'Parameters', 'Subhalo']
items if subhalos [('SubhaloCM', <HDF5 dataset "SubhaloCM": shape (1355, 3), type "<f4">), ('SubhaloGrNr', <HDF5 dataset "SubhaloGrNr": shape (1355,), type "<i4">), ('SubhaloHalfmassRad', <HDF5 dataset "SubhaloHalfmassRad": shape (1355,), type "<f4">), ('SubhaloHalfmassRadType', <HDF5 dataset "SubhaloHalfmassRadType": shape (1355, 6), type "<f4">), ('SubhaloIDMostbound', <HDF5 dataset "SubhaloIDMostbound": shape (1355,), type "<u8">), ('SubhaloLen', <HDF5 dataset "SubhaloLen": shape (1355,), type "<i4">), ('SubhaloLenType', <HDF5 dataset "SubhaloLenType": shape (1355, 6), type "<i4">), ('SubhaloMass', <HDF5 dataset "SubhaloMass": shape (1355,), type "<f4">), ('SubhaloMassInHalfRad', <HDF5 dataset "SubhaloMassInHalfRad": shape (1355,), type "<f4">), ('SubhaloMassInHalfRadType', <HDF5 dataset "SubhaloMassInHalfRadType": shape (1355, 6), type "<f4">), ('SubhaloMassInMaxRad', <HDF5 dataset "Subha

In [29]:
dataset1

array(['SubhaloCM', 'SubhaloGrNr', 'SubhaloHalfmassRad',
       'SubhaloHalfmassRadType', 'SubhaloIDMostbound', 'SubhaloLen',
       'SubhaloLenType', 'SubhaloMass', 'SubhaloMassInHalfRad',
       'SubhaloMassInHalfRadType', 'SubhaloMassInMaxRad',
       'SubhaloMassInMaxRadType', 'SubhaloMassInRad',
       'SubhaloMassInRadType', 'SubhaloMassType', 'SubhaloParent',
       'SubhaloPos', 'SubhaloSpin', 'SubhaloVel', 'SubhaloVelDisp',
       'SubhaloVmax', 'SubhaloVmaxRad'], dtype='<U24')

In [30]:
basePath = "./"

In [31]:
# fields = ['SubhaloPos']
# subhalos = il.groupcat.loadSubhalos(basePath, 99, fields=fields)
# print(np.shape(subhalos))

In [3]:
with h5py.File('/home/group10/TNGDataScripts/TNG50-4-Dark/output/snapdir_099/snap_099.2.hdf5','r') as hdf:
    ls = list(hdf.keys())
    print("list of datasets in this file: \n", ls)
    data = hdf.get('PartType1')
    dataset1 = np.array(data)
    u = data.get('Coordinates')
    print(u)
    
    print(f"{hdf.keys()}")

list of datasets in this file: 
 ['Config', 'Header', 'Parameters', 'PartType1']
<HDF5 dataset "Coordinates": shape (4915788, 3), type "<f8">
<KeysViewHDF5 ['Config', 'Header', 'Parameters', 'PartType1']>


In [33]:
dataset1

array(['Coordinates', 'ParticleIDs', 'Potential', 'SubfindDMDensity',
       'SubfindDensity', 'SubfindHsml', 'SubfindVelDisp', 'Velocities'],
      dtype='<U16')

In [34]:
hdf

<Closed HDF5 file>

In [35]:
import matplotlib as mpl
basePath = '/home/group10/TNGDataScripts/TNG50-4-Dark/output'
dm_pos = il.snapshot.loadSubset(basePath,99,'dm',['Coordinates']);
ID = il.snapshot.loadSubset(basePath,99,'dm',['ParticleIDs'])
print(np.shape(dm_pos))
print(np.shape(ID))
#plt.hist2d(dm_pos[:,0], dm_pos[:,1], norm=mpl.colors.LogNorm(), bins=64);
#plt.xlim([0,35000])
#plt.ylim([0,35000])


#plt.xlabel('x [ckpc/h]')
#plt.ylabel('y [ckpc/h]')


(19683000, 3)
(19683000,)


In [36]:
with h5py.File('/home/group10/TNGDataScripts/TNG50-4-Dark/postprocessing/offsets/offsets_099.hdf5','r') as hdf:
    ls = list(hdf.keys())
    print("list of datasets in this file: \n", ls)
    data = hdf.get('Subhalo')
    data_items = list(data.items())
    print(data_items)
    h = data.get('SnapByType')
    print(type(h))
    
    

list of datasets in this file: 
 ['FileOffsets', 'Group', 'Subhalo']
[('LHaloTree', <HDF5 group "/Subhalo/LHaloTree" (3 members)>), ('SnapByType', <HDF5 dataset "SnapByType": shape (22963, 6), type "<i8">), ('SubLink', <HDF5 group "/Subhalo/SubLink" (3 members)>)]
<class 'h5py._hl.dataset.Dataset'>


In [37]:
basePath='/home/group10/TNGDataScripts/TNG50-4-Dark/output'
SubhaloLenType = il.groupcat.loadSubhalos(basePath,99,fields=['SubhaloLenType'])

def inverseMapPartIndicesToSubhaloIDs(indsType, ptName, SubhaloLenType, debug=False, flagFuzz=True):
    """ For a particle type ptName and snapshot indices for that type indsType, compute the
        subhalo ID to which each particle index belongs. 
        If flagFuzz is True (default), particles in FoF fuzz are marked as outside any subhalo,
        otherwise they are attributed to the closest (prior) subhalo.
    """
    gcLenType = SubhaloLenType[:,1]
    with h5py.File(il.groupcat.offsetPath(basePath,99),'r') as f:
            gcOffsetsType = f['Subhalo/SnapByType'][:,1]

    # val gives the indices of gcOffsetsType such that, if each indsType was inserted
    # into gcOffsetsType just -before- its index, the order of gcOffsetsType is unchanged
    # note 1: (gcOffsetsType-1) so that the case of the particle index equaling the
    # subhalo offset (i.e. first particle) works correctly
    # note 2: np.ss()-1 to shift to the previous subhalo, since we want to know the
    # subhalo offset index -after- which the particle should be inserted
    val = np.searchsorted( gcOffsetsType - 1, indsType ) - 1
    val = val.astype('int32')

    # search and flag all matches where the indices exceed the length of the
    # subhalo they have been assigned to, e.g. either in fof fuzz, in subhalos with
    # no particles of this type, or not in any subhalo at the end of the file
    if flagFuzz:
        gcOffsetsMax = gcOffsetsType + gcLenType - 1
        ww = np.where( indsType > gcOffsetsMax[val])[0]

        if len(ww):
            val[ww] = -1

    if debug:
        # for all inds we identified in subhalos, verify parents directly
        for i in range(len(indsType)):
            if val[i] < 0:
                continue
            assert indsType[i] >= gcOffsetsType[val[i]]
            if flagFuzz:
                assert indsType[i] < gcOffsetsType[val[i]]+gcLenType[val[i]]
                assert gcLenType[val[i]] != 0

    return val

inverseMapPartIndicesToSubhaloIDs(99, 1, SubhaloLenType)

/tmp/ipykernel_4082925/1419605431.py:28: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  ww = np.where( indsType > gcOffsetsMax[val])[0]


0

In [38]:
SubhaloLenType = il.groupcat.loadSubhalos(basePath,99,fields=['SubhaloLenType'])
print(type(SubhaloLenType))

<class 'numpy.ndarray'>


In [39]:
with h5py.File('/home/group10/TNGDataScripts/TNG50-4-Dark/simulation.hdf5','r') as hdf:
    base_items = list(hdf.items())
    print("items in the base directory", base_items)
    g1 = hdf.get('Snapshots')
    g1_items = list(g1.items())
    print("items in group", g1_items)
    g2 = g1.get('99')
    g2_items = list(g2.items())
    print("items in g1", g2_items)
    halo = g2.get('PartType1')
    halo_items = list(halo.items())
    print("Halos", halo_items)
    ID = halo.get('ParticleIDs')
    dataset2= np.array(ID)

items in the base directory [('Config', <HDF5 group "/Config" (0 members)>), ('Groups', <HDF5 group "/Groups" (100 members)>), ('Header', <HDF5 group "/Header" (0 members)>), ('Offsets', <HDF5 group "/Offsets" (98 members)>), ('Parameters', <HDF5 group "/Parameters" (0 members)>), ('Snapshots', <HDF5 group "/Snapshots" (100 members)>), ('Trees', <HDF5 group "/Trees" (1 members)>)]
items in group [('0', <HDF5 group "/Snapshots/0" (2 members)>), ('1', <HDF5 group "/Snapshots/1" (2 members)>), ('10', <HDF5 group "/Snapshots/10" (2 members)>), ('11', <HDF5 group "/Snapshots/11" (2 members)>), ('12', <HDF5 group "/Snapshots/12" (2 members)>), ('13', <HDF5 group "/Snapshots/13" (2 members)>), ('14', <HDF5 group "/Snapshots/14" (2 members)>), ('15', <HDF5 group "/Snapshots/15" (2 members)>), ('16', <HDF5 group "/Snapshots/16" (2 members)>), ('17', <HDF5 group "/Snapshots/17" (2 members)>), ('18', <HDF5 group "/Snapshots/18" (2 members)>), ('19', <HDF5 group "/Snapshots/19" (2 members)>), ('2'

In [40]:
import illustris_python as il
import h5py
import numpy as np

def map_particle_index_to_parent_subhalo_index(p_index, flagFuzz=True):
    """ If flagFuzz is true, then the return has value -1 if the particle is 
    located in the FoF fuzz (outside any subhalo)."""

    basePath = '/home/group10/TNGDataScripts/TNG50-4-Dark/output'
    gName = 'Subhalo'
    snap_n = 99
    ptType = 1

    # load
    with h5py.File(il.groupcat.offsetPath(basePath,snap_n),'r') as f:
            gcOffsetsType = f['Subhalo/SnapByType'][:,ptType]

    gcLenType = il.groupcat.loadSubhalos(basePath, snap_n, 'SubhaloLenType')
    gcLenType = gcLenType[:,ptType]

    # find largest offset which is not past p_index
    val = np.searchsorted(gcOffsetsType-1,p_index)
    val = np.array([val - 1])
    val = val.astype('int32')

    if flagFuzz:
            gcOffsetsMax = gcOffsetsType + gcLenType - 1
            ww = np.where(p_index > gcOffsetsMax[val])[0]

            if len(ww):
                    val[ww] = -1

    return val

map_particle_index_to_parent_subhalo_index(dataset2)